In [ ]:
!pip install gradio pandas matplotlib

In [ ]:
import pandas as pd
import gradio as gr
import matplotlib.pyplot as plt


df = pd.read_csv("/content/airlines_flights_data.csv")


def filter_flights(source, destination, travel_class):

    data = df[
        (df["source_city"] == source) &
        (df["destination_city"] == destination) &
        (df["class"] == travel_class)
    ]

    if len(data) == 0:
        return "No flights found ❌", None

    avg_price = round(data["price"].mean(), 2)
    total_flights = len(data)

    summary = f"""
✈️ Total Flights: {total_flights}
💰 Average Price: ₹{avg_price}
"""

    plt.figure()
    data["price"].hist()
    plt.title("Price Distribution")
    plt.xlabel("Price")
    plt.ylabel("Count")

    return summary, plt.gcf()



def airline_analysis():

    airline_price = df.groupby("airline")["price"].mean()

    plt.figure()
    airline_price.plot(kind="bar")
    plt.title("Average Price by Airline")
    plt.xlabel("Airline")
    plt.ylabel("Price")

    return plt.gcf()



sources = sorted(df["source_city"].dropna().unique())
destinations = sorted(df["destination_city"].dropna().unique())
classes = sorted(df["class"].dropna().unique())



with gr.Blocks() as app:

    gr.Markdown("# ✈️ Flight Ticket Analytics Dashboard")

    with gr.Row():
        source = gr.Dropdown(sources, label="Source City")
        destination = gr.Dropdown(destinations, label="Destination City")
        travel_class = gr.Dropdown(classes, label="Class")

    btn = gr.Button("Analyze Flights")

    output_text = gr.Textbox(label="Summary")
    output_plot = gr.Plot()

    btn.click(
        filter_flights,
        inputs=[source, destination, travel_class],
        outputs=[output_text, output_plot]
    )

    gr.Markdown("## 📊 Airline Price Analysis")

    airline_btn = gr.Button("Show Airline Analysis")
    airline_plot = gr.Plot()

    airline_btn.click(
        airline_analysis,
        outputs=airline_plot
    )


app.launch(share=True)
